In [ ]:
import sys
import os

py_file_location = py_file_location = "Introducir dirección archivos.py AdapNet++"
sys.path.append(os.path.abspath(py_file_location))

import argparse
import datetime
import importlib
import os
import re
import numpy as np
import tensorflow as tf
import yaml
from dataset.helper import *

In [ ]:
num_clases='Número de clases'
max_it='Número de iteraciones'
batch_size='Tamño de lote'

In [ ]:
def get_train_batch():
    filenames = ['Dirección archivo tfrecords con imágenes y sus etiquetas']
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(lambda x: parser(x, num_clases))
    dataset = dataset.shuffle(buffer_size=100)
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(100)
    dataset = dataset.prefetch(1)
    iterator = dataset.make_one_shot_iterator()
    return iterator

def get_train_data():
    iterator = get_train_batch()
    dataA, label = iterator.get_next()
    return [dataA, label], iterator

data_list, iterator = get_train_data()

In [ ]:
module = importlib.import_module('models.'+'AdapNet_pp')
model_func = getattr(module, 'AdapNet_pp')
resnet_name = 'resnet_v2_50'
global_step = tf.Variable(0, trainable=False, name='Global_Step')

with tf.variable_scope(resnet_name):
    model = model_func(num_classes=num_clases, learning_rate=0.01,decay_steps=max_it, power=0.01 ,global_step=global_step)
    images_pl = tf.placeholder(tf.float32, [None, 384, 768, 3])
    labels_pl = tf.placeholder(tf.float32, [None, 384, 768, num_clases])
    model.build_graph(images_pl, labels_pl)
    model.create_optimizer()
 
config1 = tf.ConfigProto()
config1.gpu_options.allow_growth = True
sess = tf.Session(config=config1)
sess.run(tf.global_variables_initializer())
step = 0
total_loss = 0.0
t0 = None
import_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

path='Dirección checkpoint'

inspect_list = tf.train.list_variables(path)
names=[]

for item in inspect_list:
  if  'conv78/' in item[0] or 'conv5/' in item[0] or 'conv911/' in item[0] or 'conv912/' in item[0]:
    print('No entra en names: ', item[0])
  else:
    names.append(item[0])

initialize_variables={}
for var in import_variables:
    temp=var.name.split(':')
    if temp[0] in names:
        initialize_variables[temp[0]]=var

trainable_collection = tf.get_collection_ref(tf.GraphKeys.TRAINABLE_VARIABLES) 
for i in range(204):
  trainable_collection.pop(0)

saver=tf.train.Saver(initialize_variables)
saver.restore(sess,path)
#Número de checkpoints a guardar
saver = tf.train.Saver(max_to_keep=6000)

In [ ]:
#Lista con el valor de las pérdidas
perdidas=[]
while 1:
        try: 
            img, label = sess.run([data_list[0], data_list[1]]) 
            feed_dict = {images_pl: img, labels_pl: label}
            loss_batch, _ = sess.run([model.loss, model.train_op],feed_dict=feed_dict)

            total_loss += loss_batch
            #Se guarda un checkpoint cada 10 iteraciones
            if (step + 1) % 10 == 0:
                saver.save(sess, os.path.join('Dirección en la que se guarda el checkpoint', 'nombre del checkpoint'), step)

            if (step + 1) % 10 == 0:
                left_hours = 0

                if t0 is not None:
                    delta_t = (datetime.datetime.now() - t0).seconds
                    left_time = (max_it - step) / 10 * delta_t
                    left_hours = left_time/3600.0

                t0 = datetime.datetime.now()
                total_loss /= 10
              
                perdidas.append(total_loss)
                print ('%s %s] Step %s, lr = %f ' \
                  % (str(datetime.datetime.now()), str(os.getpid()), step,
                     model.lr.eval(session=sess)))
                print ('\t loss = %.4f' % (total_loss))
                print ('\t estimated time left: %.1f hours. %d/%d' % (left_hours, step,max_it))
                print ('\t', 'SSMA')
                total_loss = 0.0

            step += 1

            if step > max_it:
                saver.save(sess, os.path.join('Dirección en la que se guarda el checkpoint', 'nombre del checkpoint'), step-1)
                print ('training_completed')
                break

        except tf.errors.OutOfRangeError:
            print ('Epochs in dataset repeat < max_iteration')
            break